In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
submit = pd.read_csv('../input/gender_submission.csv')

In [ ]:
def mean_normalization(data):
    return (data - np.min(data))/(np.max(data)-np.min(data))

def preprocess(train): # 偷偷借用 train 來當變數
    # total 891 entries
    # Embarked only 889
    #train.head(5)
    # Sex preprocess -> one hot
    train = pd.concat([pd.get_dummies(train['Sex'], prefix='sex'), train], axis = 1)
    train.pop('Sex')
    # Name ? (先丟掉)
    # train.pop('Name')
    train['Name'] = train['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
    train['Name'] = train['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    train['Name'] = train['Name'].replace('Mlle', 'Miss')
    train['Name'] = train['Name'].replace('Ms', 'Miss')
    train['Name'] = train['Name'].replace('Mme', 'Mrs')
    train['Name'] = train['Name'].astype('category').cat.codes
    # Ticket ?
    train['Ticket_info'] = train['Ticket'].apply(lambda x : x.replace('.',"").replace('/',"").strip().split(' ')[0] if not x.isdigit() else 'X')
    train['Ticket_info'] = train['Ticket_info'].astype('category').cat.codes
    train.pop('Ticket')
    # Cabin only 204
    train['Cabin'] = train['Cabin'].fillna('NoCabin')
    train['Cabin'] = train['Cabin'].apply(lambda x: x.split()[0].strip('1234567890'))
    train['Cabin'] = train['Cabin'].astype('category').cat.codes
    #train = pd.concat([pd.get_dummies(train['Cabin']), train], axis = 1)
    #train.pop('Cabin')
    # Embarked 港口
    # train['Embarked'] = train['Embarked'].astype('category').cat.codes
    train = pd.concat([pd.get_dummies(train['Embarked'], prefix='embarked'), train], axis = 1)
    train.pop('Embarked')
    # age only 714
    # train['Age'] = train['Age'].fillna(train['Age'].mean())
    title_age_sum_dict = {}
    title_age_counter_dict = {}
    title_age_mean_dict = {}
    for title in train['Name']:
        title_age_sum_dict[title] = 0
        title_age_counter_dict[title] = 0
    for index, data in train.iterrows():
        if data['Name'] in train['Name']:
            if not pd.isna(data['Age']):
                title_age_sum_dict[data['Name']] += data['Age']
                title_age_counter_dict[data['Name']] += 1
    print(title_age_sum_dict)
    print(title_age_counter_dict)
    for title in train['Name']:
        title_age_mean_dict[title] = title_age_sum_dict[title]/title_age_counter_dict[title]
    print(title_age_mean_dict)
    mean = train['Age'].mean()
    for index, data in train.iterrows():
        if pd.isna(data['Age']):
            if pd.isna(title_age_mean_dict[data['Name']]):
                train.loc[index, 'Age'] = mean
            else:
                train.loc[index, 'Age'] = title_age_mean_dict[data['Name']]
    train['Family_size'] = train['Parch'] + train['SibSp'] + 1
    
    train['IsAlone'] = 0
    train.loc[train['Family_size'] == 1, 'IsAlone'] = 1
    # PassengerId
    train.pop('PassengerId')
    ### NAME AGE FARE TICKET_INFO -> mean_normalization
    train['Name'] = mean_normalization(train['Name'])
    train['Fare'] = mean_normalization(train['Fare'])
    train['Age'] = mean_normalization(train['Age'])
    train['Ticket_info'] = mean_normalization(train['Ticket_info'])
    train['Cabin'] = mean_normalization(train['Cabin'])
    train['Family_size'] = mean_normalization(train['Family_size'])
    train['Parch'] = mean_normalization(train['Parch'])
    train['SibSp'] = mean_normalization(train['SibSp'])
    train['Pclass'] = mean_normalization(train['Pclass'])
    
    return train

In [ ]:
x_train = preprocess(train)
x_test = preprocess(test)
#fill = np.zeros(len(x_test))
#x_test['T'] = pd.Series(fill)
y_train = x_train.pop('Survived')
y_test = submit.pop('Survived')

In [ ]:
x_train.head(20)

In [ ]:
x_test.head(1)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import regularizers
#

model = Sequential()

layer_1 = Dense(5012, activation='relu',input_dim = 15, kernel_initializer='random_uniform',bias_initializer='zeros',
               kernel_regularizer=regularizers.l2(0.0004))
dp_1 = Dropout(0.3)
layer_2 = Dense(2048, activation='relu', kernel_initializer='random_uniform',bias_initializer='zeros',
               kernel_regularizer=regularizers.l2(0.0004))
dp_2 = Dropout(0.3)
layer_3 = Dense(1024, activation='relu', kernel_initializer='random_uniform',bias_initializer='zeros',
               kernel_regularizer=regularizers.l2(0.0004))
dp_3 = Dropout(0.3)
layer_4 = Dense(500, activation='relu', kernel_initializer='random_uniform',bias_initializer='zeros',
               kernel_regularizer=regularizers.l2(0.0004))
dp_4 = Dropout(0.2)
layer_5 = Dense(500, activation='relu', kernel_initializer='random_uniform',bias_initializer='zeros',
               kernel_regularizer=regularizers.l2(0.0004))
dp_5 = Dropout(0.1)
layer_6 = Dense(100, activation='relu', kernel_initializer='random_uniform',bias_initializer='zeros',
               kernel_regularizer=regularizers.l2(0.0004))
layer_7 = Dense(1, activation='relu', kernel_initializer='random_uniform',bias_initializer='zeros',
               kernel_regularizer=regularizers.l2(0.0004))
output_layer = Dense(1, activation='sigmoid',
                    activity_regularizer=regularizers.l1(0.0004))

In [ ]:
model.add(layer_1)
model.add(dp_1)
model.add(layer_2)
model.add(dp_2)
model.add(layer_3)
model.add(dp_3)
model.add(layer_4)
model.add(dp_4)
model.add(layer_5)
model.add(dp_5)
model.add(layer_6)
model.add(layer_7)

model.add(output_layer)

In [ ]:
nadam = keras.optimizers.Nadam(lr=0.0007, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
adam = keras.optimizers.Adam(lr=0.0004, beta_1=0.9, beta_2=0.999, decay = 0.002)
model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.02, patience=20, verbose=1, mode='auto', baseline=None, restore_best_weights=False)
history = model.fit(x_train, y_train, batch_size = 104, epochs = 500, validation_split=0.112, verbose = 1, callbacks=[early_stop])

In [ ]:
x_test.head(3)
evaluate = model.evaluate(x_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

#print(history.history)
#print(dir(history.history.keys))

# 繪製訓練 & 驗證的準確值
plt.subplot(2,1,1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')

# 繪製訓練 & 驗證的損失值
plt.subplot(2,1,2)
plt.title('Model loss')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()
print("Loss: {}, Acc: {}".format(evaluate[0], evaluate[1]))

In [ ]:
predict = model.predict(x_test)

In [ ]:
predict = np.where(predict >= 0.5, 1,0)

In [ ]:
predict_train = model.predict(x_train)
predict_train = np.where(predict_train >= 0.5, 1, 0)
counter = 0
for i in range(len(predict_train)):
    if predict_train[i][0] == y_train[i]:
        counter += 1
print(counter)
print(counter/len(predict_train))

print(predict)

In [ ]:
submit['Survived'] = predict
submit.to_csv('submission.csv', index=False)


In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe
df = pd.DataFrame(np.random.randn(50, 4), columns=list('ABCD'))

# create a link to download the dataframe
create_download_link(submit)